<h1><center>OCR Project To Detect Sanskrit From A Pdf File</center></h1>

## Summary For The Notebook :

* Converts Pdf Pages To A List Of Images.
* Scans The Images.
* Detects For Possible Sanskrit Words.
* Creates a Bounding Box Around The Detected Word.
* Captures The Box And Saves It As An Image File With Extension ".jpg".
* Saves The Coordinate's Of Boxes In A Json File.
* The Box Name In The Json File Is As Follows : [box_"page-no"_"image-no"]

### Prerequisite's 
1. An Empty Json File Named "box_coordinates.json"
2. An Empty Folder Named "/box_imgs"
3. An Empty Folder Named "/pdf_imgs"

### Note :
    Make Changes In The Notebook For URL Path's Acording To Your System

### Necessary Imports

In [16]:
import pytesseract
import cv2
import os
import json
import glob
from pdf2image import convert_from_path
from PIL import Image
from tqdm import tqdm

### Pdf To Image Conversion

In [18]:
pytesseract.pytesseract.tesseract_cmd=r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
images = convert_from_path("OCR_SANSKRIT/Sanskrit_Text.pdf",65)
path = 'C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/pdf_imgs/'

In [19]:
for i in range(len(images)):   
      # Save pages as images in the pdf
    images[i].save(os.path.join(path ,'page'+ str(i) +'.jpg'), 'JPEG')

### Saving Images In List Format

In [20]:
img_list = list(map(cv2.imread,glob.glob('{}*.jpg'.format(path))))

### Appending Data To Json File

In [22]:
def box_details(name,tl,tr,bl,br):
    with open('C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/box_coordinates.json', mode='r', encoding='utf-8') as jf:
        data = json.load(jf)
        temp = data['Bounding_Boxes']
        # python object to be appended
        y = {'name': name, 'top_left': tl ,'top_right': tr ,'bottom_left': bl , 'bottom_right': br}
        # appending data to 'Bounding_Boxes'
        temp.append(y)
    write_json(data)

### Dumping Data To Json File

In [21]:
def write_json(data, filename='C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/box_coordinates.json'):
    with open(filename,'w') as f:
        json.dump(data, f, indent=4)

### Code For Generating Bounding Boxes

In [23]:
counter = 1
for i in tqdm(range(len(img_list))):
    img = img_list[i]
    hImg,wImg,_ = img.shape
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    original = img.copy()
    boxes = pytesseract.image_to_data(img, lang = 'hin')
    for x,b in enumerate(boxes.splitlines()):
        ROI_number = x
        if x!=0 :
            b = b.split()
            if len(b) == 12:
                x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                cv2.rectangle(img,(x,y),(w+x,h+y),(0,0,255),1)
                ROI = original[y:y+h, x:x+w]
                cv2.imwrite('C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/box_imgs/box_{}_{}.png'.format(counter,ROI_number), ROI)
                name = 'box_{}_{}'.format(counter,ROI_number)
                tl = '[{},{}]'.format(x , y)
                tr = '[{},{}]'.format(x+w , y)
                bl = '[{},{}]'.format(x , y+h)
                br = '[{},{}]'.format(x+w , y+h)
                box_details(name,tl,tr,bl,br)
    counter = counter + 1

100%|██████████| 5/5 [00:28<00:00,  5.73s/it]


   #### Once The Prgress Bar Is Completed Check For The Data In Respective Files

<h5><center>Disclaimer : The Engine Is Not Trained Enough For Detecting All Possible Words !</center></h5>